Le but de se code est de scrapper les prix littéraires en France entre 2019 et 2023

In [2]:
# on importe la bibliothèque requests
import requests
from bs4 import BeautifulSoup # on importe BeautifulSoup
import pandas as pd

On crée un dataframe vide avec comme colonnes les infos qu'on va importer

In [3]:
colonnes = ['Année', 'Prix', 'Auteur', 'Titre']

# Créer un DataFrame vide avec ces colonnes
df = pd.DataFrame(columns=colonnes)

print(df)

Empty DataFrame
Columns: [Année, Prix, Auteur, Titre]
Index: []


On scrape

In [4]:
def scrape_wikipedia_page_france(year):
    global df
    url = f"https://fr.wikipedia.org/wiki/Prix_litt%C3%A9raires_{year}"
    response = requests.get(url)
    response.encoding = "utf-8"  # On garantit le bon encodage

    if response.status_code == 200:  # On vérifie si la requête est un succès
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # On recherche la section France
        france_section = soup.find('h2', string="France")
        if france_section:
            # La liste <ul> qui suit contient les prix pour la France
            france_list = france_section.find_next('ul')

            if france_list:
                # On recherche tous les <li> dans la liste
                prizes = france_list.find_all('li')

                # On extrait les données
                for prize in prizes:
                    # Nom du prix
                    prize_name = prize.find('a')
                    prize_name = prize_name.text.strip() if prize_name else None

                    # Auteur
                    author = prize.find_all('a')
                    author = author[1].text.strip() if len(author) > 1 else None

                    # Titre
                    title = prize.find('i')
                    title = title.text.strip() if title else None

                    # On ajoute les résultats si les données sont complètes
                    if prize_name and author and title:
                        new_row = {'Année': year, 'Prix': prize_name, 'Auteur': author, 'Titre': title}
                        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        else:
            print(f"Section 'France' non trouvée pour l'année {year}")
    else:
        print(f"Erreur lors du scraping de l'année {year}: Status code {response.status_code}")

    return df

In [5]:
for year in range(2019, 2024):
    scrape_wikipedia_page_france(year)

In [6]:
df.head(5)

,Année,Prix,Auteur,Titre
0,2019,Prix Femina,Par les routes,Par les routes
1,2019,Prix Femina étranger,Ordesa,Ordesa
2,2019,Prix Femina essai,Emmanuelle Lambert,Giono furioso
3,2019,Prix Femina des lycéens,La Chaleur,La Chaleur
4,2019,Prix Goncourt,Tous les hommes n'habitent pas le monde de la ...,Tous les hommes n'habitent pas le monde de la ...


Création d'un fichier CSV afin de vérifier s'il n'y a pas d'incohérences

In [7]:
df.to_csv("prix_litteraires.csv", index=False)

PARTIE: Corriger les incohérences de la table

Création de fonctions

In [39]:
import requests
import re
from bs4 import BeautifulSoup as soup

# Fonction pour rechercher un livre sur Babelio et accéder à la page du premier résultat
def search_wikipedia(book_title, year):
    #reformulation du titre en différente partie
    parts = re.findall(r'\w+|[^\w\s]', book_title)

    # URL de recherche sur Livraddict
    search_title = '%20'.join(parts)
    lower_search_title = '%20'.join(parts).lower()
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    search_url = f'https://www.bing.com/search?q=Livre%20{search_title}%20{year}%20wikipedia&qs=n&form=QBRE&sp=-1&ghc=1&lq=0&pq=livre%20{lower_search_title}%20{year}%20wikipedia&sc=11-30&sk=&cvid=88409FB04E76408A93DE43207AD5027F&ghsh=0&ghacc=0&ghpl='

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    response_text = response.decode('utf-8')
    # Trouver le premier élément correspondant au lien d'un résultat
    page = BeautifulSoup(response_text, "html.parser")
    first_result = page.find('li', class_='b_algo')  # Trouve le premier résultat
    if first_result:
        link = first_result.find('a', href=True)  # Trouve le lien à l'intérieur de <a>
        if link:
            href = link['href']
            # Si c'est une URL relative, complète-la
            if href.startswith('/wiki'):
                href = 'https://fr.wikipedia.org' + href
            return href
        else:
                print("Pas de lien trouvé dans le premier résultat.")
                return None
    else:
            print("Pas de résultats trouvés.")
            return None

In [ ]:
def search_wikipedia(book_title, year):
    #reformulation du titre en différente partie
    parts = re.findall(r'\w+|[^\w\s]', book_title)

    # URL de recherche sur Livraddict
    search_title = '%20'.join(parts)
    lower_search_title = '%20'.join(parts).lower()
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    search_url = f'https://www.bing.com/search?q=Livre%20{search_title}%20{year}&qs=n&form=QBRE&sp=-1&ghc=1&lq=0&pq=livre%20{lower_search_title}%20{year}&sc=11-30&sk=&cvid=88409FB04E76408A93DE43207AD5027F&ghsh=0&ghacc=0&ghpl='

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    response_text = response.decode('utf-8')
    # Trouver le premier élément correspondant au lien d'un résultat
    page = BeautifulSoup(response_text, "html.parser")
    sponsored_results = page.find_all('li', class_='b_ad')
    
    # Parcourir les 15 premiers résultats (ou jusqu'à trouver un lien Wikipedia)
    for result in sponsored_results[:15]:  # Limiter à 15 résultats
        link = result.find('a', href=True)
        if link and "wikipedia.org" in link['href']:  # Vérifie si c'est un lien vers Wikipedia
            return link['href']
    
        else:
            print("Aucun lien Wikipedia trouvé.")
            return None

Fonction pour récupérer l'information sur l'auteur depuis la page wikipedia

In [12]:
def recup_info(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers).text
    if response == None:
        author = "problème d'URL"
    page = BeautifulSoup(response, "html.parser")
    table = page.find('table', {'class': 'infobox'})
    if table == None:
        author = "Erreur de trouvaille de la table wikipédia" 
    else: 
        rows = table.find_all('tr')
        if rows == None:
            author = "Pas de catégories trouvées dans la table"
        else:
            tr_author = rows.find("tr")
            if tr_author == None:
                author = "Pas de rubrique auteur dans la table"
            else:
                td_author = tr_author.find("td")
                if td_author == None:
                    author = "Pas de nom dans la ligne auteur (td)"
                else:
                    a_autor = td_author.find("a")
                    if a_autor == None :
                        author = "Pas de nom dans la ligne auteur (a)"
                    else:
                        author = a_autor.get("title")
                        if author == None:
                            author = "Pas de nom dans la ligne auteur (title)"
        
    return author



In [10]:
def search_author(row):
    if row["Auteur"] == row["Titre"]:
        url = search_wikipedia("Titre", "Année")
        author = recup_info(url)
        return author
    else:
        return row["Auteur"]

Fonction globale allant chercher l'auteur de l'ouvrage quand ce dernier n'est pas indiqué

In [13]:
df['type_de_document'] = df.apply(search_author, axis=1)

In [14]:
df.head(5)

,Année,Prix,Auteur,Titre,type_de_document
0,2019,Prix Femina,Par les routes,Par les routes,Erreur de trouvaille de la table wikipédia
1,2019,Prix Femina étranger,Ordesa,Ordesa,Erreur de trouvaille de la table wikipédia
2,2019,Prix Femina essai,Emmanuelle Lambert,Giono furioso,Emmanuelle Lambert
3,2019,Prix Femina des lycéens,La Chaleur,La Chaleur,Erreur de trouvaille de la table wikipédia
4,2019,Prix Goncourt,Tous les hommes n'habitent pas le monde de la ...,Tous les hommes n'habitent pas le monde de la ...,Erreur de trouvaille de la table wikipédia


METHODE 2:

In [57]:
import requests
from bs4 import BeautifulSoup

def get_answer_from_bing(question):
    # Reformulation de la question pour l'URL
    question_query = '+'.join(question.split())

    # URL de recherche Bing pour poser la question
    search_url = f"https://www.bing.com/search?q={question_query}"

    # En-têtes pour simuler une requête depuis un navigateur
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers)
    response_text = response.content.decode('utf-8')

    # Analyser la page HTML avec BeautifulSoup
    page = BeautifulSoup(response_text, "html.parser")

    # Recherche de la réponse directe dans les résultats
    snippet = page.find('div', {'class': 'b_ans'})  # Le snippet direct, souvent la première réponse

    # Si pas trouvé dans b_ans, on peut chercher dans un autre endroit comme la description du premier résultat
    if not snippet:
        # Recherche de la première description (parfois utilisée pour les réponses courtes)
        first_result_desc = page.find('li', {'class': 'b_algo'})
        if first_result_desc:
            snippet = first_result_desc.find('p')
    
    # Si un extrait de texte a été trouvé
    if snippet:
        answer = snippet.get_text(strip=True)
        return answer
    else:
        return "Aucune réponse directe trouvée."

ATTENTION, il faut entrer ce code dans le terminal: python -m spacy download fr_core_news_sm


In [56]:
def get_author(answer):
    import spacy
    if answer == "Aucune réponse directe trouvée.":
        author = "Auteur non trouvé"
    else:
        # Charger le modèle français pré-entraîné
        nlp = spacy.load("fr_core_news_sm")

        # Phrase d'exemple
        text = "Par les routes est un roman de Sylvain Prudhomme paru le 22 août 2019 aux éditions Gallimard – L'Arbalète et ayant reçu la même année le prix Femina."

        # Traiter le texte avec le modèle spaCy
        doc = nlp(text)

        # Extraire les entités nommées (NER)
        for ent in doc.ents:
            if ent.label_ == 'PER':
                author = ent.text
    return author

In [ ]:
def gen_question(title, year):
    question = "qui a écrit"+ title + "en" +year
    return question

In [60]:
def search_author(row):
    if row["Auteur"] == row["Titre"]:
        question = gen_question(row["Titre"], row["Année"])
        answer = get_answer_from_bing(question)
        author = get_author(answer)
    else:
        author = row["Auteur"]
    return author

In [61]:
df['Auteur'] = df.apply(search_author, axis=1)

TypeError: can only concatenate str (not "int") to str

TESTS

In [40]:
search_wikipedia("Par les routes", "2019")

'https://livrecritique.com/par-les-routes-2019-resume-du-roman-de-sylvain-prudhomme/'

In [17]:
recup_info('https://fr.wikipedia.org/wiki/La_Mer_des_monstres')

[<tr>
 <td class="entete universite italique" colspan="2" style="background-color:#7DA7D9;color:black;">La Mer des monstres<style data-mw-deduplicate="TemplateStyles:r160407116">.mw-parser-output .entete.universite{background-image:url("//upload.wikimedia.org/wikipedia/commons/4/42/Picto_infobox_book.png")}</style>
 </td></tr>,
 <tr><td colspan="2" style="display:none;">
 </td></tr>,
 <tr>
 <th scope="row">Auteur
 </th>
 <td><a href="/wiki/Rick_Riordan" title="Rick Riordan">Rick Riordan</a>
 </td>
 </tr>,
 <tr>
 <th scope="row">Pays
 </th>
 <td><span class="nowrap"><span class="datasortkey" data-sort-value="États unis"><span class="flagicon"><span class="mw-image-border noviewer" typeof="mw:File"><a class="mw-file-description" href="/wiki/Fichier:Flag_of_the_United_States.svg" title="Drapeau des États-Unis"><img alt="Drapeau des États-Unis" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="11" src="//upload.wikimedia.org/wikipedia/commons/th

In [16]:
def recup_info(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers).content
    page = BeautifulSoup(response, "html.parser")
    table = page.find('table', {'class': 'infobox'})
    rows = table.find_all('tr')
    return rows